In [ ]:
# -*- coding: utf-8 -*-
"""
@author: Rukang Xu
"""

In [1]:
import sys
print(sys.version)

In [2]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Notes for package installation:
> When specifying `-e` or `develop`, MMDetection is installed on **dev** mode , any local modifications made to the code will take effect without reinstallation.

In [5]:
# install pytorch with specified version based on cuda110 and available mmcv-full source for cuda110 and pytorch1.7.0
!pip install '../input/cellis/mmdetection_cuda110/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '../input/cellis/mmdetection_cuda110/torchvision-0.8.0-cp37-cp37m-linux_x86_64.whl' --no-deps
import torch, torchvision
torch_version = torch.__version__
print(torch.__version__, torch.cuda.is_available())


In [6]:
# check the gpu device being used
if torch.cuda.is_available():
  print('the device name is:')
  print(torch.cuda.get_device_name(torch.cuda.current_device()))
  print('Memory Usage:')
  print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
  print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
else:
  print('No gpu device is available.')

In [7]:

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==1.3.18 -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html

# Install mmdetection
# !rm -rf mmdetection
# !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection
!rm -rf CBNetV2
!git clone https://github.com/VDIGPKU/CBNetV2.git 
%cd CBNetV2


!pip install -e .

# install pandas to deal with current dataset
!pip install pandas

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow

# install opencv
!pip install opencv-python

# install pyyaml
!pip install pyyaml

# install pathlib2, an alternative to os.path
!pip install pathlib2

# install tdam which displays nice progress bars, ipywidgets for tqdm's notebook support
!pip install tqdm
!pip install ipywidgets

# install sklearn
!pip install scikit-learn

In [8]:
# check TensorFlow installation when it is needed
# import tensorflow as tf
# tf.test.gpu_device_name()
# Standard output is '/device:GPU:0'

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# import other useful python packages
import pathlib2
import os
print(f'the current work dir is: {os.getcwd()}')
import time
import os.path as osp
import PIL
import numpy as np
import pandas as pd
import mmcv
import matplotlib.pyplot as plt
import cv2

## Download a pretrained model to the checkpoints
1. backbone:
  - swin transformer
    - https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_base_patch4_window7_224_22k.pth from [reference link](https://github.com/VDIGPKU/CBNetV2/issues/45)
    - https://github.com/CBNetwork/storage/releases/download/v1.0.0/cascade_mask_rcnn_cbv2_swin_small_patch4_window7_mstrain_400-1400_adamw_3x_coco.pth.zip

## Macros

In [9]:
from easydict import EasyDict as edict

__C = edict()
local_cfg = __C


__C.USE_GOOGLE_DRIVE = False
__C.CURRENT_DIR = pathlib2.Path.cwd()
print("current dir: ", __C.CURRENT_DIR.as_posix())
# the highest level dir that holds all relevant files
__C.PROJECT_DIR = __C.CURRENT_DIR.parent
print("project dir: ", __C.PROJECT_DIR.as_posix())
__C.DATA_DIR = __C.PROJECT_DIR.parent / 'cellisdata'
print("useful dataset dir: ", __C.PROJECT_DIR.as_posix())

__C.SPLIT_IN_THREE = True
__C.TEST_SIZE = 0.3
__C.VAL_SIZE = 1/float(6)
__C.RANDOM_STATE = 42
__C.FEATURES = ['id', 'annotation', 'width', 'height', 'cell_type']

__C.HEIGHT, __C.WIDTH = 520, 704

## Train a detector on LIVECell dataset and dataset for competition

To train a new detector, there are usually three things to do:
1. Support a new dataset
2. Modify the config
3. Train a new detector



### Support a new dataset

There are three ways to support a new dataset in MMDetection: 
  1. reorganize the dataset into COCO format.
  2. reorganize the dataset into a middle format.
  3. implement a new dataset.

Usually we recommend to use the first two methods which are usually easier than the third.

In this tutorial, we gives an example that converting the data into the format of existing datasets like COCO, VOC, etc. Other methods and more advanced usages can be found in the [doc](https://mmdetection.readthedocs.io/en/latest/tutorials/new_dataset.html#).

#### Check the dataset

In [10]:
# Check the working directory

# Install tree first
# TODO this following origin file may be not general to each case
!apt-get -q install tree
!tree /kaggle/working/ -L 1

In [11]:

!cp -r /kaggle/input/apexforcellis/apex /kaggle/working/
%cd /kaggle/working/apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd /kaggle/working/CBNetV2
import os 
os.getcwd()

In [12]:
!mkdir -p /kaggle/working/CBNetV2/checkpoints/
!cp /kaggle/input/cellisckpt/cascade_mask_rcnn_cbv2_swin_small_patch4_window7_mstrain_400-1400_adamw_3x_coco.pth /kaggle/working/CBNetV2/checkpoints/

### Prepare a config

In [13]:
import os
os.getcwd()

In [14]:
# Copy the config scripts - base and child for cbnetv2 swin tiny to the local 

#!cp /kaggle/input/cellisconfig/cascade_mask_rcnn_cbv2_swin_small_patch4_window7_mstrain_400-1400_adamw_2x_live_2.py /kaggle/working/CBNetV2/configs/cbnet
!cp /kaggle/input/cellisconfig/cascade_mask_rcnn_cbv2_swin_small_patch4_window7_mstrain_400-1400_adamw_2x_live_2_tiny.py /kaggle/working/CBNetV2/configs/cbnet

In [17]:
!rm -rf /kaggle/working/CBNetV2/live_version_swin_small

In [40]:
# copy the usefule data for training into the specified place according to config file

!mkdir -p ../data/livecell_images/
!cp -RT /kaggle/input/cellisdata/ /kaggle/working/data/livecell_images/
#!mv /kaggle/working/data/cellisdata /kaggle/working/data/livecell_images

In [42]:
!ls /kaggle/working/

In [41]:
# check if the fiel structure of  /kaggle/working/data/  is right!
!ls /kaggle/working/data/livecell_images

**Resume from the checkpoint if needed**

In [ ]:
'''
!cp "$ds_path_pre"/epoch_3.pth "$local_cfg.PROJECT_DIR"/CBNetV2
from mmcv import Config
cfg = Config.fromfile('configs/cbnet/cascade_mask_rcnn_cbv2_swin_small_patch4_window7_mstrain_400-1400_adamw_2x_live_2.py')
cfg.keys()
cfg.resume_from = './epoch_3.pth'
'''

In [43]:
#%%writefile ./configs/livecell/mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_livecell.py
!tools/dist_train.sh configs/cbnet/cascade_mask_rcnn_cbv2_swin_small_patch4_window7_mstrain_400-1400_adamw_2x_live_2_tiny.py 1

**Extract the essential model checkpoint and log and download it**

In [ ]:
!mkdir ./live_version1_extract

In [ ]:
!cp ./live_version1/{20211223_111435.log,20211223_111435.log.json,epoch_24.pth,latest.pth} ./live_version1_extract  

In [ ]:
!zip -r ./live_version1_extract.zip ./live_version1_extract 

### Competition dataset

In [ ]:
# Copy the data part for competition
print('Start copying extra data from Google Drive to Virtual Machine...')
tic_move = time.time()  
!cp "$ds_path_pre"/competition_coco_test.json "$local_cfg.PROJECT_DIR"/data/livecell_images/
!cp "$ds_path_pre"/competition_coco_train.json "$local_cfg.PROJECT_DIR"/data/livecell_images/
!cp "$ds_path_pre"/competition_coco_val.json "$local_cfg.PROJECT_DIR"/data/livecell_images/
!cp "$ds_path_pre"/train.zip "$local_cfg.PROJECT_DIR"/data/livecell_images/
!cp "$ds_path_pre"/cascade_mask_rcnn_cbv2_swin_tiny_patch4_window7_mstrain_480-800_adamw_2x_compet.py "$local_cfg.PROJECT_DIR"/data/livecell_images/
print(('Done (t={:0.2f}s)'.format(time.time() - tic_move)))

In [ ]:
!ls "$local_cfg.PROJECT_DIR"/data/livecell_images/

In [ ]:
!cp "$local_cfg.PROJECT_DIR"/data/livecell_images/cascade_mask_rcnn_cbv2_swin_tiny_patch4_window7_mstrain_480-800_adamw_2x_compet.py "$local_cfg.PROJECT_DIR"/CBNetV2/configs/cbnet

In [ ]:
!unzip "$local_cfg.PROJECT_DIR"/data/livecell_images/train.zip -d "$local_cfg.PROJECT_DIR"/data/livecell_images/

**Check the validity of annotation files w.r.t competition**

In [ ]:
ann_dir = local_cfg.PROJECT_DIR / 'data' / 'livecell_images'
val_ann_file = ann_dir / 'competition_coco_val.json'
val_ann_file

In [ ]:
file_format = str(val_ann_file).split('.')[-1]
val_data_infos = mmcv.load(str(val_ann_file), file_format=file_format)
val_data_infos.keys()

#### Start training

In [ ]:
!tools/dist_train.sh configs/cbnet/cascade_mask_rcnn_cbv2_swin_tiny_patch4_window7_mstrain_480-800_adamw_2x_compet.py 1

Log analysis

In [ ]:
# draw bbox_mAP and segm_mAP respectively

!python tools/analysis_tools/analyze_logs.py plot_curve compet_version1/20211223_143913.log.json --keys bbox_mAP --legend bbox_mAP --out bbox_mAP_cascade_mask_rcnn_cbv2_swin_tiny_ds_tiny.pdf

!python tools/analysis_tools/analyze_logs.py plot_curve compet_version1/20211223_143913.log.json --keys segm_mAP --legend mask_mAP --out segm_mAP_cascade_mask_rcnn_cbv2_swin_tiny_ds_tiny.pdf

**Store the selected checkpoint(s)**

In [ ]:
!mkdir ./compet_version1_extract

In [ ]:
!cp ./compet_version1/{20211223_133806.log,20211223_143913.log,20211223_143913.log.json,epoch_11.pth} ./compet_version1_extract

In [ ]:
!zip -r ./compet_version1_extract.zip ./compet_version1_extract

**Save result to Google Drive**

In [ ]:
!cp "$local_cfg.PROJECT_DIR"/CBNetV2/compet_version1_extract.zip "$ds_path_pre"/

In [ ]:
from google.colab import files
path_to_file = str(local_cfg.PROJECT_DIR/'CBNetV2'/'compet_version1_extract.zip')
files.download(path_to_file)

**Evaluate the model**

In [ ]:
!python tools/test.py configs/cbnet/cascade_mask_rcnn_cbv2_swin_tiny_patch4_window7_mstrain_480-800_adamw_2x_compet.py compet_version1/latest.pth --eval bbox segm

In [ ]:
# Copy the results above here
[('bbox_mAP', 0.144), 
('bbox_mAP_50', 0.313), 
('bbox_mAP_75', 0.113), 
('bbox_mAP_s', 0.148), 
('bbox_mAP_m', 0.174), 
('bbox_mAP_l', 0.005), 
('bbox_mAP_copypaste', '0.144 0.313 0.113 0.148 0.174 0.005'), 
('segm_mAP', 0.121), 
('segm_mAP_50', 0.303), 
('segm_mAP_75', 0.068), 
('segm_mAP_s', 0.119), 
('segm_mAP_m', 0.157), 
('segm_mAP_l', 0.154), 
('segm_mAP_copypaste', '0.121 0.303 0.068 0.119 0.157 0.154')]